![tracker](https://us-central1-vertex-ai-mlops-369716.cloudfunctions.net/pixel-tracking?path=statmike%2Fvertex-ai-mlops%2FDev%2Fnew&file=Unified+Content+Holder.ipynb)
<!--- header table --->
<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/statmike/vertex-ai-mlops/blob/main/Dev/new/Unified%20Content%20Holder.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo">
      <br>Run in<br>Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https%3A%2F%2Fraw.githubusercontent.com%2Fstatmike%2Fvertex-ai-mlops%2Fmain%2FDev%2Fnew%2FUnified%2520Content%2520Holder.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo">
      <br>Run in<br>Colab Enterprise
    </a>
  </td>      
  <td style="text-align: center">
    <a href="https://github.com/statmike/vertex-ai-mlops/blob/main/Dev/new/Unified%20Content%20Holder.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      <br>View on<br>GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/statmike/vertex-ai-mlops/main/Dev/new/Unified%20Content%20Holder.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      <br>Open in<br>Vertex AI Workbench
    </a>
  </td>
</table>

# Notes

## Readme.md files

start with header: vertex-ai-mlops/folder/readme.md

all notebooks need a line like this after main header in first cell:

> From the [Working With Document AI](https://github.com/statmike/vertex-ai-mlops/blob/main/Working%20With%20Document%20AI/readme.md) series in the [vertex-ai-mlops](https://github.com/statmike/vertex-ai-mlops/blob/main/readme.md) repository.

---
## Snippets

In [ ]:
import logging

# setup logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logger.addHandler(logging.StreamHandler())

logging.info('Finished setting up clients')

In [ ]:
try:
    # setting the limit_per_minute to 80, higher than the actual limit of 60
    responses = await study_notes(vocab_words[0:80], limit_per_minute = 80)
except Exception as err:
    print(f"{type(err).__name__} was raised: {err}")

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import vertexai
import vertexai.vision_models # Imagen Models
import vertexai.language_models # PaLM and Codey Models
import vertexai.generative_models # for Gemini Models

vertexai.init(project = PROJECT_ID, location = REGION)

In [ ]:
# Gemini Models
gemini_text = vertexai.generative_models.GenerativeModel("gemini-1.0-pro")
gemini_multimodal = vertexai.generative_models.GenerativeModel("gemini-1.0-pro-vision")

# PaLM and Codey Models
text_model = vertexai.language_models.TextGenerationModel.from_pretrained('text-bison')
chat_model = vertexai.language_models.ChatModel.from_pretrained('chat-bison')
textembed_model = vertexai.language_models.TextEmbeddingModel.from_pretrained('textembedding-gecko')
codegen_model = vertexai.language_models.CodeGenerationModel.from_pretrained('code-bison')
codecomp_model = vertexai.language_models.CodeGenerationModel.from_pretrained('code-gecko')
codechat_model = vertexai.language_models.CodeChatModel.from_pretrained('codechat-bison')

# Imagen Models
imagecap_model = vertexai.vision_models.ImageCaptioningModel.from_pretrained("imagetext")
imageqna_model = vertexai.vision_models.ImageQnAModel.from_pretrained("imagetext")
imagetext_model = vertexai.vision_models.ImageTextModel.from_pretrained("imagetext")
multimodalembed_model = vertexai.vision_models.MultiModalEmbeddingModel.from_pretrained('multimodalembedding')
imagen1_model = vertexai.preview.vision_models.ImageGenerationModel.from_pretrained('imagegeneration@002')
imagen2_model = vertexai.preview.vision_models.ImageGenerationModel.from_pretrained('imagegeneration@005')

In [ ]:
prediction = textgen_model.predict('Create is a short summary of the rules of baseball?')

print(prediction.text)

In [ ]:
prediction = gemini_text.generate_content('Create is a short summary of the rules of baseball?')

print(prediction.text)

In [49]:
MODEL_ID = 'gemini-1.5-pro-preview-0409'
#MODEL_ID = 'gemini-experimental'
request = dict(
    contents = dict(
        role = 'user',
        parts = dict(
            text = prompt_baseball
        )
    ),
    generationConfig = dict(
        temperature = 0.0,
        topP = .8,
        topK = 25,
        maxOutputTokens = 500,
        responseMimeType = 'text/plain'
    )
)
token = !gcloud auth application-default print-access-token
headers = {
    "content-type": "application/json",
    "Authorization": f'Bearer {token[0]}'
}

response = requests.post(
    f'https://{REGION}-aiplatform.googleapis.com/v1/projects/{PROJECT_ID}/locations/{REGION}/publishers/google/models/{MODEL_ID}:generateContent',
    #data = json.dumps(dict(instances = [request])),
    data = json.dumps(request),
    headers = headers
)

response_text = json.loads(response.text)['candidates'][0]['content']['parts'][0]['text']
print(response_text)

The bases in baseball are 15 inches (38.1 cm) square. ⚾️ 



Request preditiction but note that the Python SDK does not (yet) include the batch prediction methods:

In [409]:
import google.auth
creds, project_id = google.auth.default()

In [416]:
request = dict(
    displayName = "gemini_batch_api",
    model = "publishers/google/models/gemini-1.0-pro-002",
    inputConfig = dict(
        instancesFormat = "bigquery",
        bigquerySource = dict(inputUri = f"bq://{table.project}.{table.dataset_id}.{table.table_id}")
    ),
    outputConfig = dict(
        predictionsFormat = "bigquery",
        bigqueryDestination = dict(outputUri = f"bq://{table.project}.{table.dataset_id}.{table.table_id}-output")
    )
)

In [417]:
creds.refresh(google.auth.transport.requests.Request())
response = requests.post(
    f"https://{REGION}-aiplatform.googleapis.com/v1/projects/{PROJECT_ID}/locations/{REGION}/batchPredictionJobs",
    headers = {"authorization": f"Bearer {creds.token}", "Content-Type": "application/json; charset=utf-8"},
    data = json.dumps(request)
)

In [418]:
response.json()

{'name': 'projects/1026793852137/locations/us-central1/batchPredictionJobs/5855598329345343488',
 'displayName': 'gemini_batch_api',
 'model': 'publishers/google/models/gemini-1.0-pro-002',
 'inputConfig': {'instancesFormat': 'bigquery',
  'bigquerySource': {'inputUri': 'bq://statmike-mlops-349915.applied_genai.getting-started_gemini_batch'}},
 'outputConfig': {'predictionsFormat': 'bigquery',
  'bigqueryDestination': {'outputUri': 'bq://statmike-mlops-349915.applied_genai.getting-started_gemini_batch-output'}},
 'state': 'JOB_STATE_PENDING',
 'createTime': '2024-07-29T21:17:57.303159Z',
 'updateTime': '2024-07-29T21:17:57.303159Z',
 'modelVersionId': '1'}

In [ ]:
from google.api_core import retry
@retry.Retry(timeout=100)
    def

- https://cloud.google.com/python/docs/reference/storage/1.39.0/retry_timeout
- https://github.com/GoogleCloudPlatform/generative-ai/blob/eec279b6c3065aacc5e7539da36f9d15bbf504df/search/retrieval-augmented-generation/examples/rag_google_documentation.ipynb#L407
- https://googleapis.dev/python/google-api-core/latest/retry.html

In [ ]:
from deepdiff import DeepDiff
DeepDiff(dict_a, dict_b)

In [ ]:
import copy
hold_files_pages = copy.deepcopy(files_pages)

In [ ]:
%%bigquery
CREATE SCHEMA IF NOT EXISTS `statmike-mlops-349915.feature_data_architecture`
    OPTIONS(
        location = 'US'
    )

In [ ]:
%%bigquery
CREATE OR REPLACE TABLE feature_data_architecture.features_eav (
  entity_id STRING,
  feature_name STRING,
  feature_value STRUCT<string_value STRING, int64_value INT64, date_value DATE>,
  feature_datatype STRING
);

In [ ]:
%%bigquery features_eav_schema
SELECT 
 TO_JSON_STRING(
    ARRAY_AGG(STRUCT(
      column_name AS name,
      data_type AS type)
    ORDER BY ordinal_position), TRUE) AS schema
FROM
  feature_data_architecture.INFORMATION_SCHEMA.COLUMNS
WHERE
  table_name = 'features_eav'

In [ ]:
# cancel BQ job with SQL
CALL BQ.JOBS.CANCEL('ed57cb8b-be40-442f-bcf1-9edb00081c10')

---
## Colab Setup

When running this notebook in [Colab](https://colab.google/) or [Colab Enterprise](https://cloud.google.com/colab/docs/introduction), this section will authenticate to GCP (follow prompts in the popup) and set the current project for the session.

In [1]:
PROJECT_ID = 'statmike-mlops-349915' # replace with project ID

In [11]:
try:
    from google.colab import auth
    auth.authenticate_user(project_id = PROJECT_ID)
    print('Colab authorized to GCP')
except Exception:
    print('Not a Colab Environment, Continue.')
    pass

Not a Colab Environment, Continue.


---
## Installs

The list `packages` contains tuples of package import names and install names.  If the import name is not found then the install name is used to install quitely for the current user.

In [ ]:
# tuples of (import name, install name, min_version)
packages = [
    ('google.cloud.aiplatform', 'google-cloud-aiplatform', '1.38.0'),
    ('google.cloud.', 'google-cloud-')
]

import importlib
install = False
for package in packages:
    if not importlib.util.find_spec(package[0]):
        print(f'installing package {package[1]}')
        install = True
        !pip install {package[1]} -U -q --user
    elif len(package) == 3:
        if importlib.metadata.version(package[0]) < package[2]:
            print(f'updating package {package[1]}')
            install = True
            !pip install {package[1]} -U -q --user

## API Enablement

In [ ]:
!gcloud services enable aiplatform.googleapis.com

### Restart Kernel (If Installs Occured)

After a kernel restart the code submission can start with the next cell after this one.

In [12]:
if install:
    import IPython
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)
    IPython.display.display(IPython.display.Markdown("""<div class=\"alert alert-block alert-warning\">
        <b>⚠️ The kernel is going to restart. Please wait until it is finished before continuing to the next step. The previous cells do not need to be run again⚠️</b>
        </div>"""))

---
## Setup

Inputs

In [1]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'statmike-mlops-349915'

In [2]:
REGION = 'us-central1'
EXPERIMENT = 'embeddings'
SERIES = 'applied-genai'

# specify a GCS Bucket
GCS_BUCKET = PROJECT_ID

Packages

In [229]:
import vertexai.language_models
import vertexai.vision_models
from google.cloud import bigquery

import os
import pygbif
import requests
import asyncio
import multiprocessing
import tqdm
import time
import IPython.display 

import docker

Clients

In [4]:
# vertex ai clients
vertexai.init(project = PROJECT_ID, location = REGION)

# bigquery client
bq = bigquery.Client(project = PROJECT_ID)

Docker Check:

In [18]:
docker_client = docker.from_env()

if docker_client.ping():
    print(f"Docker is installed and running. Version: {docker_client.version()['Version']}")
else:
    print('Docker is either not installed or not running - please fix before proceeding.')

Docker is installed and running. Version: 20.10.17


parameters:

In [205]:
DIR = f"temp/{EXPERIMENT}"

In [10]:
SERVICE_ACCOUNT = !gcloud config list --format='value(core.account)' 
SERVICE_ACCOUNT = SERVICE_ACCOUNT[0]
SERVICE_ACCOUNT

'1026793852137-compute@developer.gserviceaccount.com'

List the service accounts current roles:

In [11]:
!gcloud projects get-iam-policy $PROJECT_ID --filter="bindings.members:$SERVICE_ACCOUNT" --format='table(bindings.role)' --flatten="bindings[].members"

ROLE
roles/bigquery.admin
roles/owner
roles/run.admin
roles/secretmanager.secretAccessor
roles/storage.objectAdmin


environment:

In [211]:
if not os.path.exists(DIR):
    os.makedirs(DIR)